<a href="https://colab.research.google.com/github/yoohyunseok/Machine-Learnig-Deep-Learning/blob/main/5_3EnsembleLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 정형 데이터와 비 정형 데이터
- 정형 데이터: 어떠한 구조로 가지런히 정리되어 있는 데이터로, CSV, DB, excel에 저장하기 쉬움
- 비 정형 데이터: 정형 데이터와 반대로 DB나 Excel로 표현하기 어려운 데이터 ex) 텍스트 데이터, 사진, 음악 등
- 비 정형 데이터는 머신러닝으로는 다루기 까다롭기 때문에 신경망 알고리즘으로 주로 다룬다

#앙상블 학습(ensemble learning)
- 정형 데이터를 다루는데 가장 뛰어난 성능을 내는 머신러닝 알고리즘
- 대부분 결정 트리 기반으로 만들어져 있음

# 랜덤 포레스트(Random Forest)
- 대표적인 앙상블 학습 알고리즘으로 성능이 안정적임
- 랜덤하게 선택한 샘플과 특성을 사용하므로 과대 적합 방지
- 대략적인 작동 방식
    1. 결정 트리를 랜덤하게 만들어 결정트리의 숲을 만듦
    2. 그 후 각 결정 트리의 예측을 사용해 최종 예측을 만듦

- 훈련 데이터 만드는 방식(Bootstrap)
  - 입력한 훈련 데이터에서 훈련 데이터와 동일한 개수의 샘플을 랜덤하게 추출하여 훈련 데이터를 만듦. **이때 샘플의 중복 추출을 허용함**
  - 이렇게 만들어진 샘플을 Bootstrap sample 이라고 부른다.

- 각 노드를 분할할 때 특성 선택
  - 분류(RandomForestClassifier): 전체 특성 개수의 제곱근 만큼의 특성을 무작위로 선택 ex) 전체 특성이 4개면 그중 2개를 임의로 고른다
  - 회귀(RandomForestRegressor): 전체 특성을 사용

- 예측 방식
  - 분류: 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼는다
  - 회귀: 각 트리의 예측을 평균

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state = 42)

In [12]:
#cross validation 수행
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1)
#return_train_score = True => 훈련 세트에 대한 점수도 반환
#과대 적합 판단할 때 유용함
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [13]:
rf.fit(train_input, train_target) #원래 훈련 데이터로 합쳐서 다시 모델 훈련
print(rf.feature_importances_) #특성 중요도(각 결정 트리의 특성 중요도 취합한것)

[0.23167441 0.50039841 0.26792718]


***5***-1장 결정 트리의 특성 중요도와 비교했을 때 당도의 중요도가 감소하고 알콜 도수와 pH의 중요도가 올라감
why? 랜덤 포레스트가 특성의 일부를 랜덤하게 선택해서 훈련했기 때문이다. ***그 결과 과대 적합을 줄이고 일반화 성능을 높이는데 도움이 된다***

In [14]:
#obb(out of bag) sample을 이용한 모델 평가
#obb sample은 bootstrap 과정에서 bootstrap sample에 포함되지 않고 남는 샘플을 의미한다
#이 obb sample을 검증 세트의 역할로 사용하면, 결과적으로 교차 검증보다 훈련 세트에 더 많은 샘플을 사용할 수 있다.
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


# 엑스트라 트리(Extra Tree)
- 기본적으로 랜덤 포레스트와 비슷함. 100개의 결정트리를 훈련하고, 결정 트리가 제공하는 대부분의 매개변수를 지원하고, 전체 특성 중 일부 특성을 랜덤하게 골라서 노드 분할
- But bootstrap sample을 사용 X.
- 각 결정 트리를 만들 때 전체 훈련 세트를 사용
- 대신에 노드를 분할할 때 ***가장 좋은 분할을 찾지 않고 무작위로 분할***(과대 적합 감소)
- ***그래서 훈련 속도가 빠르지만, 더 많은 트리가 필요함***
- 이전 장의 splitter = 'random'을 사용하는 결정 트리를 사용한다.

In [15]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))


0.9974503966084433 0.8887848893166506


In [16]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


이녀석도 5-1장의 결정 트리보다 당도에 대한 의존성이 낮아진걸 볼 수 있음

# 그레이디언트 부스팅(Gradient boosting)
- 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블하는 방법
- sklearn의 GradientBoostingClassifier는 기본으로 깊이가 3인 결정트리 100개를 사용
- 깊이가 얕은 결정 트리를 사용하기에 ***과대 적합에 강하고 높은 일반화 성능을 기대할 수 있다.***
- 경사 하강법을 사용하여 트리를 앙상블에 추가한다.
  - 분류: 로지스틱 손실 함수
  - 회귀: 평균 제곱 오차 함수(mse)
  - 경사 하강법은 손실 함수가 가장 작은 곳으로 내려가는 것이라서 깊이가 3밖에 안되는 얕은 결정 트리를 사용함!
- 일반적으로 랜덤 포레스트 보다 조금 더 높은 성능을 얻을 수 있지만, ***트리를 순서대로 추가해서 훈련 속도가 느림***

## 평균 제곱 오차(Mean Squared Error, MSE)

평균 제곱 오차(MSE)는 회귀 모델의 성능을 평가하는 데 널리 사용되는 지표입니다.

### 개념

- **오차**: 모델의 예측값과 실제 정답값 사이의 차이입니다. (예측값 - 실제값)
- **제곱**: 오차를 제곱하여 음수 값을 양수로 만들고, 큰 오차에 더 큰 페널티를 부여합니다.
- **평균**: 모든 오차 제곱 값들의 평균을 계산하여 전체적인 모델의 예측 정확도를 나타냅니다.

### 공식

MSE = $\frac{1}{n} \sum_{i=1}^{n}(Y_i - \hat{Y}_i)^2$

- $n$: 데이터 포인트의 개수
- $Y_i$: $i$번째 실제 값
- $\hat{Y}_i$: $i$번째 예측 값

### 특징

1. **큰 오차에 대한 민감성**: 오차를 제곱하기 때문에 큰 오차에 대해 더 민감하게 반응하여, 이상치(outlier)가 있을 경우 MSE 값이 크게 증가할 수 있습니다.
2. **단위**: 종속 변수의 단위 제곱으로 나타나기 때문에 실제 값의 단위와 다릅니다. 이 문제를 해결하기 위해 MSE의 제곱근인 RMSE(Root Mean Squared Error)를 사용하기도 합니다.
3. **목표**: 회귀 모델을 훈련할 때 MSE를 최소화하는 방향으로 모델의 파라미터가 최적화됩니다. 즉, 모델의 예측값이 실제값에 최대한 가깝도록 만듭니다.

In [17]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#과대 적합을 잘 억제하는 모습

0.8881086892152563 0.8720430147331015


In [18]:
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2, random_state = 42)
#n_estimators = 500 => 결정 트리 개수 500개로 지정, learning_rate = 0.2 => 학습률 0.2로 지정(기본값 = 0.1)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#결정 트리 개수가 5배로 늘어나도 과대적합을 잘 억제함

0.9464595437171814 0.8780082549788999


In [19]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15887763 0.6799705  0.16115187]


subsample 매개변수(기본값 = 1)
- 트리 훈련에 사용할 훈련 세트의 비율을 정하는 매개변수
- subsample의 값이 1보다 작으면 훈련 세트의 일부를 사용한다(like 확률적/미니배치 경사하강법)

# 히스토그램 기반 그레이디언트 부스팅(Histogram-based Gradient Boosting)
- 입력 특성을 256개의 구간으로 나누기 때문에 노드를 분할할 때 최적의 분할을 ***매우 빠르게 찾을 수 있음***
- 256개의 구간 중에서 하나를 떼어놓고 누락된 값을 위해서 사용하기 때문에 입력에 누락된 값이 있어도 따로 전처리가 필요하지 않음

In [23]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#과대 적합을 잘 억제 + gradient boosting 보다 조금 더 높은 성능

0.9321723946453317 0.8801241948619236


In [24]:
#특성 중요도 계산 in train set
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state = 42, n_jobs = -1) #n_reapeat: 특성을 랜덤하게 섞을 횟수 지정
#permutation_importance()는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화화는지를 관찰하영 어떤 특성이 중요한지를 계산
#반환하는 객체에는 특성 중요도(importances), 평균(importances_mean), 표준 편차(importances_std)를 담고있음
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [25]:
#특성 중요도 계산 in test set
result = permutation_importance(hgb, test_input, test_target, n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)
#gradient boosting과 비슷하게 약간 더 당도에 집중
# 이를 통해 모델을 실전에 투입했을 때 어떤 특성에 관심을 둘지 예상 가능

[0.05969231 0.20238462 0.049     ]


In [27]:
hgb.score(test_input, test_target)

0.8723076923076923

**XGboost 라이브러리로 Histogram based gradient boosting  진행**

In [28]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state = 42) #tree_method = 'hist'로 지정해서 histogram based gradient boosting
scores = cross_validate(xgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#

0.9567059184812372 0.8783915747390243


**LightGBM 라이브러리로 Histogram based gradient boosting 진행**

In [30]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state = 42)
scores = cross_validate(lgb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) #훈련 세트, 검증 세트 점수 출력

0.935828414851749 0.8801251203079884
